In [1]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch
from langchain_openai import ChatOpenAI

# Load environment variables from .env
load_dotenv()

# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4o-mini")

In [2]:
# Define plantillas de prompt para diferentes tipos de retroalimentación
plantilla_retroalimentacion_positiva = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un asistente servicial."),
        ("human",
         "Genera una nota de agradecimiento para esta retroalimentación positiva: {retroalimentacion}."),
    ]
)

plantilla_retroalimentacion_negativa = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un asistente servicial."),
        ("human",
         "Genera una respuesta que aborde esta retroalimentación negativa: {retroalimentacion}."),
    ]
)

plantilla_retroalimentacion_neutral = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un asistente servicial."),
        (
            "human",
            "Genera una solicitud de más detalles para esta retroalimentación neutral: {retroalimentacion}.",
        ),
    ]
)

plantilla_escalar_retroalimentacion = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un asistente servicial."),
        (
            "human",
            "Genera un mensaje para escalar esta retroalimentación a un agente humano: {retroalimentacion}.",
        ),
    ]
)

In [3]:
# Define la plantilla de clasificación de retroalimentación
plantilla_clasificacion = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un asistente servicial."),
        ("human",
         "Clasifica el sentimiento de esta retroalimentación como positiva, negativa, neutral o escalar: {retroalimentacion}."),
    ]
)


In [6]:
# Define the runnable branches for handling feedback
branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        plantilla_retroalimentacion_positiva | model | StrOutputParser()  # Positive feedback chain
    ),
    (
        lambda x: "negative" in x,
        plantilla_retroalimentacion_negativa| model | StrOutputParser()  # Negative feedback chain
    ),
    (
        lambda x: "neutral" in x,
        plantilla_retroalimentacion_neutral | model | StrOutputParser()  # Neutral feedback chain
    ),
    plantilla_escalar_retroalimentacion | model | StrOutputParser()
)

In [7]:
# Create the classification chain
classification_chain = plantilla_clasificacion | model | StrOutputParser()

In [8]:
# Combine classification and response generation into one chain
chain = classification_chain | branches

In [10]:
reseña = "El producto es terrible. Se rompió después de un solo uso y la calidad es muy mala."
resultado = chain.invoke({"retroalimentacion": reseña})

# Imprimir el resultado
print(resultado)


Asunto: Escalación de Retroalimentación Negativa

Hola [Nombre del Agente o Equipo],

Espero que te encuentres bien. Quiero hacerte llegar una retroalimentación que hemos recibido recientemente, la cual tiene un sentimiento negativo. Es importante que la revisemos para entender mejor las preocupaciones del cliente y buscar una solución adecuada.

Detalles de la retroalimentación:
- [Incluir breve descripción de la retroalimentación o el problema mencionado]

Te agradecería que le des prioridad a este asunto y lo analices para poder tomar las acciones necesarias. Si necesitas más información o contexto, no dudes en decírmelo.

Gracias por tu atención a este tema.

Saludos,
[Tu Nombre]  
[Tu Puesto]  
[Tu Información de Contacto]  


In [11]:
reseña = "El producto es excelente. Disfruté mucho usarlo y lo encontré muy útil."
resultado = chain.invoke({"retroalimentacion": reseña})

# Imprimir el resultado
print(resultado)

Asunto: Escalación de retroalimentación positiva

Hola [Nombre del agente humano o equipo],

Espero que te encuentres bien. Quiero escalar una retroalimentación que he recibido, la cual se clasifica como **positiva**. 

[Incluir detalles específicos sobre la retroalimentación, como quién la dio, qué aspectos fueron destacados, etc. Esto ayudará a brindar contexto y a resaltar la importancia de la retroalimentación.]

Creo que es importante reconocer este tipo de comentarios y considerar cómo podemos seguir mejorando a partir de ellos. 

Gracias por tu atención y quedo a tu disposición para cualquier pregunta.

Saludos,  
[Tu nombre]  
[Tu posición]  
[Tu contacto]  


In [12]:
reseña = "El producto está bien. Funciona como se espera, pero no es nada excepcional."
resultado = chain.invoke({"retroalimentacion": reseña})

# Imprimir el resultado
print(resultado)


Asunto: Solicitud de Más Detalles sobre su Retroalimentación

Estimado/a [Nombre del Cliente],

Gracias por tomarse el tiempo de compartir su opinión sobre nuestro producto. Valoramos mucho sus comentarios y queremos asegurarnos de que podamos seguir mejorando.

Hemos notado que su retroalimentación se clasifica como neutral. Apreciamos que haya mencionado que el producto funciona como se espera, pero también nos gustaría entender mejor su comentario sobre que no es nada excepcional. 

Para poder ofrecerle una experiencia más satisfactoria, ¿podría proporcionarnos más detalles sobre los aspectos que considera que podrían mejorarse? Cualquier información adicional que pueda compartir nos ayudará a identificar áreas clave para el desarrollo futuro de nuestros productos.

Agradecemos su colaboración y su interés en ayudarnos a mejorar.

Atentamente,

[Tu Nombre]  
[Tu Puesto]  
[Tu Empresa]  
[Tu Información de Contacto]


In [13]:
reseña = "No estoy seguro sobre el producto todavía. ¿Puedes decirme más sobre sus características y beneficios?"
resultado = chain.invoke({"retroalimentacion": reseña})

# Imprimir el resultado
print(resultado)

Asunto: Solicitud de Más Detalles sobre la Retroalimentación Recibida

Estimado/a [Nombre del remitente],

Espero que te encuentres bien. He recibido tu retroalimentación y agradezco que te hayas tomado el tiempo para compartir tus pensamientos. Sin embargo, he notado que tu comentario se clasifica como neutral y expresa cierta incertidumbre.

Para poder comprender mejor tu perspectiva y abordar cualquier inquietud que puedas tener, me gustaría solicitar más detalles sobre los puntos específicos que consideras relevantes. ¿Podrías ampliar sobre las áreas donde sientes que se necesita más información o aclaración? Esto me ayudará a ofrecerte una respuesta más completa y a mejorar en los aspectos que mencionas.

Agradezco de antemano tu colaboración y espero tu respuesta.

Saludos cordiales,

[Tu nombre]  
[Tu cargo]  
[Tu información de contacto]  
